# How to store transactions for Anthropic chat in PromptSail

This guide illustrates how to store transactions for Anthropic Claude models using the [Langchain Python SDK](https://python.langchain.com/docs/integrations/chat/anthropic/). It directly interfaces with the Anthropic API Claude 3 family models.

To begin, you'll need an account on [Anthropic](https://anthropic.com/). If you don't have one, create it. 

Next, obtain your API key 🔑 from the [Anthropic console](https://console.anthropic.com/settings/keys).

Then, paste this API key into the `.env` file located in the `examples` folder and assign it to the `ANTHROPIC_API_KEY` variable.


Install all the necessary packages from [examples/pyproject.toml](pyproject.toml) by running the following command:

```bash 
cd prompt_sail/examples
poetry install
```

In [1]:
from langchain_anthropic import ChatAnthropic
from dotenv import dotenv_values
from rich import print  


config = dotenv_values(".env")

anthropic_key = config["ANTHROPIC_API_KEY"]
print(
    f"Anthropic api key={anthropic_key[0:13]}...{anthropic_key[-3:]}"
)

Anthropic api key=sk-ant-api03-...gAA

Models comparsion you can find at [Anthropic documentation](https://docs.anthropic.com/claude/docs/models-overview#model-comparison).

## This part is just about testing if the API key is correct and you can connect straight to the Anthropic API

In [2]:
from langchain_core.prompts import ChatPromptTemplate

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

In [3]:
#the smallest model 
model_name = "claude-3-haiku-20240307"

# # middle size model
# model_name = "claude-3-sonnet-20240229"
# # the largest model
#model_name = "claude-3-opus-20240229"


chat = ChatAnthropic(temperature=0, anthropic_api_key=anthropic_key, model_name=model_name)

In [4]:
chain = prompt | chat
resp = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "text": "I love Python",
    }
)

print(resp)

AIMessage(
    content='Ich liebe Python.',
    response_metadata={
        'id': 'msg_01FCiTcnRgmNUu8kGgSmT94i',
        'model': 'claude-3-haiku-20240307',
        'stop_reason': 'end_turn',
        'stop_sequence': None,
        'usage': {'input_tokens': 22, 'output_tokens': 8}
    },
    id='run-e1a75d9a-08bf-4437-b6dd-f74c5f030772-0'
)

## Create a request to the Anthropic API via PromptSail proxy



* Go to demo [PromptSail](https://try-promptsail.azurewebsites.net/) and create a new project or use an existing one.
* [Run the PromptSail docker images](https://promptsail.com/docs/quick-start-guide/#pull-and-run-the-docker-images-from-ghcr) and go to UI at http://localhost/.
We will have to setup a project and add ai provider. 


Create new project with you `project_slug`or edit existing one for purpose of this example we will use `model-playground`.

Add your own Anthropic provider by editing the project settings and click "Add AI Provider" button, this will create the mapping between the Anthropic endpoint to promptsail proxy URL. 

In the form set the `api base url` to your Anthropic endpoint to: **'https://api.anthropic.com/v1'**
 
and add meaningfull `deployment name`.

Save the changes by clicking "Add" and "Update" button.
 
In mongo it will create new entry in `ai_providers` array, similar to this one:

```bash
{
     ai_providers: [
        {
            deployment_name: 'anth claude'
            slug: 'anth-claude',
            api_base: 'https://api.anthropic.com/',
            description: '',
            provider_name: 'Anthropic'
        }
    ],
}
```

In this case we will use the default `model-playground` settings:
* with project_slug -> 'model-playground' 
* slugified deployment_name -> 'anth-claude'

resulting in promptsail proxy url like this: 

**http://localhost:8000/models-playground/anth-claude/** or

**https://try-promptsail.azurewebsites.net/api/models-playground/anth-claude/**

In [5]:
ps_api_base = "http://localhost:8000/models-playground/anth-claude/"
ps_api_base_try = "https://try-promptsail.azurewebsites.net/api/models-playground/anth-claude/"
ps_api_base_try_tagged = "https://try-promptsail.azurewebsites.net/api/models-playground/anth-claude/?tags=media,copy&target_path="


#the smallest model 
model_name = "claude-3-haiku-20240307"

# # middle size model
# model_name = "claude-3-sonnet-20240229"
# # the largest model
#model_name = "claude-3-opus-20240229"

ps_chat = ChatAnthropic(
    temperature=0, 
    anthropic_api_key=anthropic_key, 
    model_name=model_name,
    anthropic_api_url=ps_api_base
)

In [6]:
ps_chain = prompt | ps_chat
resp = ps_chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "text": "I love Machine learning",
    }
)

print(resp)

AIMessage(
    content='Ich liebe maschinelles Lernen.',
    response_metadata={
        'id': 'msg_01FcAPW39ZUUQSmDh5hfbPaz',
        'model': 'claude-3-haiku-20240307',
        'stop_reason': 'end_turn',
        'stop_sequence': None,
        'usage': {'input_tokens': 23, 'output_tokens': 13}
    },
    id='run-328aa29e-05b4-4006-abf1-c0d759a8485e-0'
)